In [1]:
import ads
import base64
import numpy as np
import tensorflow as tf

from ads.common.model_metadata import UseCaseType
from ads.model.framework.tensorflow_model import TensorFlowModel
from io import BytesIO
from uuid import uuid4

ads.set_auth(auth="resource_principal")

# Create the TensorFlow Model
mnist = tf.keras.datasets.mnist
(trainx, trainy), (testx, testy) = mnist.load_data()
trainx, testx = trainx / 255.0, testx / 255.0

model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10),
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
model.fit(trainx, trainy, epochs=1)

2025-12-04 18:18:09.111747: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-04 18:18:09.112979: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


1875/1875 [==============================] - 9s 4ms/step - loss: 0.2948 - accuracy: 0.9139


In [2]:
# Exercise/validate the prediction
model.predict(testx[0:1])

array([[-2.097159  , -4.5638795 ,  2.0069768 ,  2.4260192 , -4.5270343 ,
        -1.8234626 , -9.919938  , 11.495073  , -2.7603838 ,  0.74910516]],
      dtype=float32)

In [3]:
## TENSORFLOW FRAMEWORK SERIALIZATION
# Create an ADS model
tensorflow_model = TensorFlowModel(estimator=model, artifact_dir=f"./model-artifact-{str(uuid4())}")

# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Available     Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Not Available Local tested .predict from score.py                               
save()    Not Available Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  Not Available Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [4]:
# Prepare the ADS Model
tensorflow_model.prepare(
    X_sample=trainx,
    y_sample=trainy,
    use_case_type=UseCaseType.MULTINOMIAL_CLASSIFICATION,
    force_overwrite=True,
)

# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Available     Local tested .predict from score.py                               
save()    Available     Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  Not Available Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [5]:
# Verify the ADS model
tensorflow_model.verify(testx)

# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

Start loading model.h5 from model directory /home/datascience/demo/model-artifact-48d16bd4-d728-4436-812f-e0e7bec71b86 ...
Model is successfully loaded.


Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done          Local tested .predict from score.py                               
save()    Available     Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  Not Available Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [6]:
# Save the ADS model
model_id = tensorflow_model.save()

# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

Start loading model.h5 from model directory /home/datascience/demo/model-artifact-48d16bd4-d728-4436-812f-e0e7bec71b86 ...
Model is successfully loaded.
['output_schema.json', 'score.py', 'model.h5', 'runtime.yaml']


loop1:   0%|          | 0/5 [00:00<?, ?it/s]

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done          Local tested .predict from score.py                               
save()    Done          Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  Available     Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [7]:
# Deploy and create an endpoint for the TensorFlow model
tensorflow_model.deploy(
    display_name="DIY TensorFlow Model For MNIST Classification",
    #deployment_log_group_id="ocid1.loggroup.oc1.xxx.xxxxx",
    #deployment_access_log_id="ocid1.log.oc1.xxx.xxxxx",
    #deployment_predict_log_id="ocid1.log.oc1.xxx.xxxxx",
    # Shape config details mandatory for flexible shapes:
    deployment_instance_shape="VM.Standard.E4.Flex",
    deployment_ocpus=1,
    deployment_memory_in_gbs=16,
)
print(f"Endpoint: {tensorflow_model.model_deployment.url}")

# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

loop1:   0%|          | 0/6 [00:00<?, ?it/s]

Endpoint: https://modeldeployment.us-chicago-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.us-chicago-1.amaaaaaakoygqqiacvqaoqmotns6jwhr74bybznpwq76276ghp2q4fegxznq


Actions Needed
Step      Status    Details                                                           
initiate  Done      Initiated the model                                               
prepare() Done      Generated runtime.yaml                                            
                    Generated score.py                                                
                    Serialized model                                                  
                    Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done      Local tested .predict from score.py                               
save()    Done      Conducted Introspect Test                                         
                    Uploaded artifact to model catalog                                
deploy()  ACTIVE    Deployed the model                                                
predict() Available Called deployment predict endpoint

In [8]:
# Generate prediction by invoking the deployed endpoint
tensorflow_model.predict(testx[:3])['prediction']

[[-2.097158670425415,
  -4.563878536224365,
  2.00697660446167,
  2.4260194301605225,
  -4.527033805847168,
  -1.8234622478485107,
  -9.919939041137695,
  11.495071411132812,
  -2.7603843212127686,
  0.7491049766540527],
 [0.5165860056877136,
  0.04259549453854561,
  9.878021240234375,
  4.97014856338501,
  -11.67444896697998,
  0.8579947352409363,
  0.29563701152801514,
  -11.526954650878906,
  0.12496829777956009,
  -10.014646530151367],
 [-4.53732442855835,
  6.108939170837402,
  0.29487666487693787,
  -1.6181063652038574,
  -2.4118921756744385,
  -2.1029810905456543,
  -1.1176451444625854,
  0.7167274951934814,
  -1.2741578817367554,
  -3.053781270980835]]

In [9]:
# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

Actions Needed
Step      Status    Details                                                           
initiate  Done      Initiated the model                                               
prepare() Done      Generated runtime.yaml                                            
                    Generated score.py                                                
                    Serialized model                                                  
                    Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done      Local tested .predict from score.py                               
save()    Done      Conducted Introspect Test                                         
                    Uploaded artifact to model catalog                                
deploy()  ACTIVE    Deployed the model                                                
predict() Available Called deployment predict endpoint

In [10]:
# Prepare data sample for prediction and save it to file 'data-payload'
data = testx[:3]
np_bytes = BytesIO()
np.save(np_bytes, data, allow_pickle=True)
data = base64.b64encode(np_bytes.getvalue()).decode("utf-8")
with open('data-payload', 'w') as f:
    f.write('{"data": "' + data + '", "data_type": "numpy.ndarray"}')

In [11]:
# Alternately, the OCI CLI may be invoked with the command line
print("Use the following commands in the ODSC terminal to invoke the deployment via the OCI CLI...\n")
print("$ cd /home/datascience/demo")
print("$ oci raw-request --http-method POST --target-uri " + tensorflow_model.model_deployment.url + "/predict --request-body file://data-payload")

Use the following commands in the ODSC terminal to invoke the deployment via the OCI CLI...

$ cd /home/datascience/demo
$ oci raw-request --http-method POST --target-uri https://modeldeployment.us-chicago-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.us-chicago-1.amaaaaaakoygqqiacvqaoqmotns6jwhr74bybznpwq76276ghp2q4fegxznq/predict --request-body file://data-payload
